# Progetto Finale Data Science - Travel

**Il tuo compito sarà seguire ogni punto e commentarlo, mostrandomi di aver capito cosa tu stia facendo. Se alcuni punti si rivelano impossibili spiegami la motivazione. Troverai probabilmente alcuni modelli o tecniche che non hai propriamente studiato. Non preoccuparti, la sperimentazione, la scoperta e il continuo aggiornamento fanno parte del gioco. CANCELLA QUESTO MESSAGGIO PRIMA DI CARICARE IL PROGETTO**

In questo progetto si analizzerà un dataset sulla soddisfazione dei clienti di una compagnia aerea.

l'obiettivo è **poter prevedere la soddisfazione degli utenti in base a dei parametri sulla soddisfazione dei servizi della compagnia aerea**

## EDA (Exploratory data analysis)

### DATA SELECTION

in questo progetto verrano caricati due dataset, uno per il training ed uno per il test, fornito dalla traccia su start2impact:

saranno caricati tutte le librerie python che verranno utilizzate in questo progetto:

In [1]:
import pandas as pd
from google.colab import files
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython.display import display

pd.set_option('display.max_columns', None)



S'importa i due csv forniti dalla traccia su start2impact. E' stato utilizzato il codice sotto perchè per la creazione del progetto è stato utilizzato con la piattaforma cloud di google Colab:

In [ ]:
uploaded = files.upload()

In [ ]:
df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')

In [ ]:
df_train.shape, df_test.shape

si puoi analizzare che sommando la grandezza dei 2 dataset, il training set rappresenta il 75% dei dati mentre il training set rappresenta il 25% dei dati.

Viene dato un primo sguardo ai dati del dataset di training

In [ ]:
display(df_train.head(), df_test.head())

In [ ]:
display(df_train.info(), df_test.info())

viene dato uno sguardo alle colonne presenti:

In [ ]:
display(df_train.columns, df_test.columns)

viene dato uno sguardo ai valori unici presenti in ogni colonna:

In [ ]:
display(df_train.nunique())
display("valori unici nel test set: ")
display(df_test.nunique())



In [ ]:
for colonna in df_train.columns:
    valori_unici = df_train[colonna].unique()
    print(f"Valori presenti nella variabile '{colonna}' unici: {valori_unici}")

### DATA CLEANING

Dopo aver dato un primo sguardo al dataset, si può proseguire con il data cleaning:

In questa fase verranno applicate modifiche sia al training che al test set, quindi su tutto il dataset, per vari motivi:
1.  **Errate interpretazioni delle relazioni tra variabili**: La presenza di dati sporchi o mancanti nel training set può portare a distorcere le relazioni tra le variabili.

2.   **Visualizzazioni fuorvianti**: Dati non puliti possono generare grafici che non riflettono accuratamente le distribuzioni e le tendenze reali dei dati, portando a interpretazioni errate.

3.   **Destabilizzazione dei modelli**: Dati sporchi possono compromettere la stabilità dei modelli di apprendimento. Questo può portare a overfitting (modello troppo complesso che si adatta ai rumori del training set) o underfitting (modello troppo semplice che non cattura i pattern dei dati), riducendo la robustezza e la capacità di generalizzazione dei modelli

# PER IL MOMENTO NON CANCELLARE QUESTO TESTO PIU' IL TESTO INERENTE AL TEST SET, QUANDO UTILIZZERAI IL TEST SET PER FARE LA PREDIZIONE CAPISCI SE FUNZIONA








#### DATA REDUCTION

Si procede a eliminare le variabili non indispensabili o ridondanti in tutto il dataset come:

1. **Unnamed: 0**: Questa variabile assegna un ID a ogni riga. Tuttavia, utilizzando pandas, viene automaticamente creato un indice per ogni riga, rendendo questa variabile ridondante.
2. **id**: Questa variabile non è indispensabile per il progetto, in quanto non contribuisce all'analisi o alla costruzione del modello



In [ ]:
df_train_feat_clean= df_train.drop(columns=["Unnamed: 0", "id"], axis=1)
df_test_feat_clean= df_test.drop(columns=["Unnamed: 0", "id"], axis=1)

df_train_feat_clean

In [ ]:
df_test_feat_clean

### DATA CLEANING (DEVI CANCELLARE IL TITOLO DI SOPRA)

Si procede ad analizzare se sono presenti dei valori nulli (isnull()) o NaN (isna()), verrà effettuato un calcolo in percentuale per fare più chiarezza sui dati mancanti:

s'inizia con il test set :

In [ ]:
def calculate_null_percentage(df):
  nan_df = (df.isna().sum() / len(df) ) * 100
  null_df = (df.isnull().sum() / len(df) ) * 100
  return nan_df, null_df

In [ ]:
nan_df_test, null_df_test =  calculate_null_percentage(df_test_feat_clean)

nan_df_test, null_df_test


È emerso che valori nulli o NaN sono presenti solo nella variabile *Arrival Delay in Minutes* e rappresentano quasi lo **0.32%** dei valori NaN e nulli, percentuali veramente basse

si analizza il training set:

In [ ]:
nan_df_train, null_df_train =  calculate_null_percentage(df_train_feat_clean)

nan_df_train, null_df_train

Anche nel training set, la variabile *Arrival Delay in Minutes* è l'unica variabile che ha valori nulli o NaN, i quali rappresentano quasi lo **0.30%** dei valori, una percentuale alquanto bassa. Si analizzano righe con valori nulli:

In [ ]:

null_value = df_test_feat_clean.isna()
df_test_missing_val = df_test_feat_clean[null_value.any(axis=1)]

null_value = df_train_feat_clean.isna()
df_train_missing_val = df_train_feat_clean[null_value.any(axis=1)]

display(df_test_missing_val, df_train_missing_val)


È emerso che i valori nulli sono pochi rispetto al numero totale di valori nella variabile, sia nel training set che nel test set. Pertanto, si potrebbe considerare l'eliminazione delle righe contenenti valori nulli in entrambi i dataset, poiché la perdita di dati non influenzerebbe in modo significativo l'integrità dei dataset. Si utilizzerà il metodo *dropna()* su entrambi i dataset:

In [ ]:
df_train_clean = df_train_feat_clean.dropna()
df_test_clean = df_test_feat_clean.dropna()


Si effettua una controprova per stabilire che tutti i valori nulli sono stati eliminati:

In [ ]:
nan_df_test, null_df_test =  calculate_null_percentage(df_test_clean)
nan_df_test, null_df_test

In [ ]:
nan_df_train, null_df_train =  calculate_null_percentage(df_test_clean)

nan_df_train, null_df_train

I due dataset sono completamente puliti dai valori nulli

### DATA EXPLORATION


In questa fase verranno rintracciati gli **outliers**. Gli outliers sono dei dati diversi dagli altri che si discostano da una distribuzione generale di una variabile o di un insieme di variabili. Essi possono distorcere le stime dei parametri del modello di apprendimento e possono influenzare le statistiche descrittive (come la media e la deviazione standard) portando a interpretazioni errate dei dati.


Prima di iniziare ad analizzare il dataset, è necessario decidere quale dataset utilizzare o se utilizzare entrambi. In questa fase è stato deciso di utilizzare solo il training set perchè se si togliessero gli outliers anche nel test set, si creerebbe un accuretezza falsata perchè si creerebbe un test set ottimale e pulito e l'obiettivo è lasciare i dati di test che assomigliano quanto possibile a quelli che si riceverebbero in un ambiente di produzione.

Gli outliers possono essere rintracciati sia da un punto di vista grafico o calcolarlo con i quartili recuperati dal metodo *describe()*. Verranno analizzati da un punto di vista grafico.

In [ ]:
df = pd.melt(df_train_clean, value_vars=['Gender', 'Customer Type', 'Age', 'Type of Travel',
       'Class', 'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes'], var_name='Variabile', value_name='Valore')

fig, axs = plt.subplots(nrows=11, ncols=2, figsize=(12, 30))
axs = axs.flatten()
for i, ax in enumerate(axs):
    if i < len(df_train_clean.columns):
        sns.boxplot(data=df[df['Variabile'] == df_train_clean.columns[i]], ax=ax)
        ax.set_title(df_train_clean.columns[i])
    else:
        ax.axis('off')

plt.tight_layout()
plt.show()


Osservando i boxplot molte variabili non presentano dei outliers

In [ ]:
df_train_clean.describe()

In [ ]:
df_train_clean.hist(figsize=(10, 8))
plt.tight_layout()
plt.show()

In [ ]:
df_train_feat_clean_subset = df_train_feat_clean.loc[: , "Inflight wifi service":"Cleanliness"]
sns.boxplot(data=df_train_feat_clean_subset)

plt.xticks(rotation=90)

plt.show()

In [ ]:
df_train_feat_clean_subset = df_train_feat_clean.loc[: , ["Departure Delay in Minutes", "Arrival Delay in Minutes"]]
sns.boxplot(data=df_train_feat_clean_subset)

plt.xticks(rotation=90)

plt.show()